### Go to this given URL and solve the following questions
>URL: https://www.youtube.com/@PW-Foundation/videos

### Q1. Write a python program to extract the video URL of the first five videos.

### Q2. Write a python program to extract the URL of the video thumbnails of the first five videos.


#### Q3. Write a python program to extract the title of the first five videos.
#### Q4. Write a python program to extract the number of views of the first five videos.
### Q5. Write a python program to extract the time of posting of video for the first five videos.

>> Get the Channel name

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
import time

# from webelement_highlighter import WebElementHighlighter
from selenium.webdriver.support.ui import WebDriverWait
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

URL = "https://www.youtube.com/@PW-Foundation/videos"

driver.get(URL)
driver.maximize_window()

driver.find_element(By.XPATH, '//*[@id="tabsContent"]/tp-yt-paper-tab[7]/div').click()
time.sleep(0.5)
driver.find_element(By.XPATH, '//*[@id="tabsContent"]/tp-yt-paper-tab[7]/div').click()
time.sleep(0.5)
next_url = driver.current_url
driver.get(next_url)
# print(next_url)
time.sleep(3)
driver.find_element(By.ID, 'channel-info').click()
time.sleep(3)
channel_url = str(driver.current_url).split('/')[-1]

print(channel_url)


time.sleep(3)
driver.delete_all_cookies()
driver.close()

# Get details from all data 

In [ ]:
from googleapiclient.discovery import build
scopes = ["https://www.googleapis.com/auth/youtube.readonly"]
import pandas as pd
api_key = secretAPI
api_service_name = "youtube"
api_version = "v3"
client_secrets_file = client.json


youtube = build('youtube','v3', developerKey=api_key)

# Accessing the playlist id
playlist_id= channel_url

# we need create function to get video id to access the video detail 

def get_video_ids(youtube,playlist_id):
    
    request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId=playlist_id,
                maxResults = 50)
    response=request.execute()
    
    video_ids=[]
    
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
## while loop to check if we have more pages than 50 
        
    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part='contentDetails',
                        playlistId=playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
            response=request.execute()
            
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
                
            next_page_token = response.get('nextPageToken')
            
    return (video_ids)
    
video_ids=get_video_ids(youtube,playlist_id)
# fot play url
video_url = []
for name in video_ids:
    urlname = f"https://www.youtube.com/watch?v={name}"
    video_url.append(urlname)

def get_video_details(youtube, video_ids):
    all_video_stats = []
    
    for i in range(0, len(video_ids), 50): ## processing 50 videos at a time as # limitation of youtube-we can fetch maxm 50 video ids at a time
        request = youtube.videos().list(
                part='snippet,statistics',
                id=','.join(video_ids[i:i+50]))
        response = request.execute()
        a = 1
        try:
            for video in response['items']:
                # print(video['statistics'])
                # if a == 1:
                #     break
                video_stats = dict(Title = video['snippet']['title'],
                                Published_date = video['snippet']['publishedAt'],
                                Views = video['statistics']['viewCount'],
                                Likes = video['statistics']['likeCount'],
                                Comments = video['statistics']['commentCount']
                                )
                all_video_stats.append(video_stats)
        except Exception as e:
            print(e)
    
    return all_video_stats

video_details=get_video_details(youtube, video_ids)

video_data = pd.DataFrame(video_details)
print(video_data)